# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [3]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's. It is important to sort the unique values. Otherwise, the method is creating different results after each restart of the notebook. 

In [5]:
doi_counted = doi.value_counts().sort_index(ascending=True)
doi_counted

10.1001/jamainternmed.2020.1369       1
10.1001/jamanetworkopen.2020.16382    1
10.1001/jamanetworkopen.2020.17521    1
10.1001/jamanetworkopen.2020.20485    1
10.1001/jamanetworkopen.2020.24984    1
                                     ..
10.9745/ghsp-d-20-00115               1
10.9745/ghsp-d-20-00171               1
10.9745/ghsp-d-20-00218               1
10.9758/cpn.2020.18.4.607             1
10.9781/ijimai.2020.02.002            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API. (https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4)

In [6]:
#Adapted from https://github.com/ElsevierDev/elsapy/blob/master/exampleProg.py
def AffiliationsFromScopusByDOI(client, doi):
    """obtain additional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [7]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [8]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [9]:
return_example = AffiliationsFromScopusByDOI(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [10]:
df_current_extra_info = pd.DataFrame()
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    df_current_extra_info
except:
    print("The DataFrame is empty")
    #if the dataframe is not empty set the variable to show the dataframe

The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [11]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

10776

In [12]:
#df_current_extra_info

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [ ]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < len_dois: ####################################################################### -> upto modified, normally len_dois
    print("Position fetched: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position fetched: 10776 -> 10.1007/s11547-020-01236-5
Position fetched: 10777 -> 10.1007/s11547-020-01269-w
Position fetched: 10778 -> 10.1007/s11547-020-01271-2
Position fetched: 10779 -> 10.1007/s11547-020-01272-1
Position fetched: 10780 -> 10.1007/s11547-020-01279-8
Position fetched: 10781 -> 10.1007/s11547-020-01291-y
Position fetched: 10782 -> 10.1007/s11547-020-01293-w
Position fetched: 10783 -> 10.1007/s11547-020-01296-7
Position fetched: 10784 -> 10.1007/s11547-020-01298-5
Position fetched: 10785 -> 10.1007/s11547-020-01305-9
Position fetched: 10786 -> 10.1007/s11547-020-01307-7
Position fetched: 10787 -> 10.1007/s11547-020-01311-x
Position fetched: 10788 -> 10.1007/s11547-020-01312-w
Position fetched: 10789 -> 10.1007/s11547-020-01313-9
Position fetched: 10790 -> 10.1007/s11547-020-01320-w
Position fetched: 10791 -> 10.1007/s11547-020-01327-3
Position fetched: 10792 -> 10.1007/s11547-020-01328-2
Position fetched: 10793 -> 10.1007/s11548-020-02171-6
Position fetched: 10794 -> 1

Position fetched: 10929 -> 10.1007/s11606-020-06474-x
Position fetched: 10930 -> 10.1007/s11606-020-06476-9
Position fetched: 10931 -> 10.1007/s11606-020-06478-7
Position fetched: 10932 -> 10.1007/s11606-020-06481-y
Position fetched: 10933 -> 10.1007/s11606-020-06484-9
Position fetched: 10934 -> 10.1007/s11606-020-06485-8
Position fetched: 10935 -> 10.1007/s11606-020-06489-4
Position fetched: 10936 -> 10.1007/s11606-020-06491-w
Position fetched: 10937 -> 10.1007/s11606-020-06503-9
Position fetched: 10938 -> 10.1007/s11606-020-06518-2
Position fetched: 10939 -> 10.1007/s11606-020-06527-1
Position fetched: 10940 -> 10.1007/s11606-020-06536-0
Position fetched: 10941 -> 10.1007/s11606-020-06547-x
Position fetched: 10942 -> 10.1007/s11606-020-06557-9
Position fetched: 10943 -> 10.1007/s11606-020-06565-9
Position fetched: 10944 -> 10.1007/s11606-020-06567-7
Position fetched: 10945 -> 10.1007/s11606-020-06578-4
Position fetched: 10946 -> 10.1007/s11606-021-06598-8
Position fetched: 10947 -> 1

Position fetched: 11082 -> 10.1007/s11695-020-04741-8
Position fetched: 11083 -> 10.1007/s11695-020-04804-w
Position fetched: 11084 -> 10.1007/s11695-020-04813-9
Position fetched: 11085 -> 10.1007/s11695-020-04820-w
Position fetched: 11086 -> 10.1007/s11695-020-04829-1
Position fetched: 11087 -> 10.1007/s11695-020-04830-8
Position fetched: 11088 -> 10.1007/s11695-020-04866-w
Position fetched: 11089 -> 10.1007/s11695-020-04874-w
Position fetched: 11090 -> 10.1007/s11695-020-04876-8
Position fetched: 11091 -> 10.1007/s11695-020-04883-9
Position fetched: 11092 -> 10.1007/s11695-020-04913-6
Position fetched: 11093 -> 10.1007/s11695-020-04976-5
Position fetched: 11094 -> 10.1007/s11695-020-05004-2
Position fetched: 11095 -> 10.1007/s11695-020-05007-z
Position fetched: 11096 -> 10.1007/s11695-020-05044-8
Position fetched: 11097 -> 10.1007/s11695-020-05054-6
Position fetched: 11098 -> 10.1007/s11695-020-05059-1
Position fetched: 11099 -> 10.1007/s11695-020-05083-1
Position fetched: 11100 -> 1

Position fetched: 11235 -> 10.1007/s11845-020-02355-8
Position fetched: 11236 -> 10.1007/s11845-020-02356-7
Position fetched: 11237 -> 10.1007/s11845-020-02361-w
Position fetched: 11238 -> 10.1007/s11845-020-02375-4
Position fetched: 11239 -> 10.1007/s11845-020-02389-y
Position fetched: 11240 -> 10.1007/s11845-020-02400-6
Position fetched: 11241 -> 10.1007/s11845-020-02405-1
Position fetched: 11242 -> 10.1007/s11845-020-02407-z
Position fetched: 11243 -> 10.1007/s11845-020-02415-z
Position fetched: 11244 -> 10.1007/s11845-020-02433-x
Position fetched: 11245 -> 10.1007/s11845-020-02442-w
Position fetched: 11246 -> 10.1007/s11845-020-02450-w
Position fetched: 11247 -> 10.1007/s11845-020-02455-5
Position fetched: 11248 -> 10.1007/s11845-020-02459-1
Position fetched: 11249 -> 10.1007/s11845-020-02460-8
Position fetched: 11250 -> 10.1007/s11845-020-02465-3
Position fetched: 11251 -> 10.1007/s11845-020-02468-0
Position fetched: 11252 -> 10.1007/s11845-020-02474-2
Position fetched: 11253 -> 1

Position fetched: 11388 -> 10.1007/s11999-010-1341-5
Position fetched: 11389 -> 10.1007/s12008-020-00745-x
Position fetched: 11390 -> 10.1007/s12010-014-0749-x
Position fetched: 11391 -> 10.1007/s12010-018-2804-5
Position fetched: 11392 -> 10.1007/s12010-020-03475-8
Position fetched: 11393 -> 10.1007/s12011-012-9379-8
Position fetched: 11394 -> 10.1007/s12011-015-0470-9
Position fetched: 11395 -> 10.1007/s12011-020-02358-7
Position fetched: 11396 -> 10.1007/s12011-020-02377-4
Position fetched: 11397 -> 10.1007/s12011-020-02496-y
Position fetched: 11398 -> 10.1007/s12011-020-02546-5
Position fetched: 11399 -> 10.1007/s12012-020-09603-4
Position fetched: 11400 -> 10.1007/s12012-020-09621-2
Position fetched: 11401 -> 10.1007/s12013-012-9505-4
Position fetched: 11402 -> 10.1007/s12013-014-0084-4
Position fetched: 11403 -> 10.1007/s12013-016-0743-8
Position fetched: 11404 -> 10.1007/s12013-017-0821-6
Position fetched: 11405 -> 10.1007/s12013-018-0839-4
Position fetched: 11406 -> 10.1007/s12

Position fetched: 11541 -> 10.1007/s12038-020-00046-1
Position fetched: 11542 -> 10.1007/s12038-020-00053-2
Position fetched: 11543 -> 10.1007/s12038-020-00069-8
Position fetched: 11544 -> 10.1007/s12038-020-00102-w
Position fetched: 11545 -> 10.1007/s12038-020-00114-6
Position fetched: 11546 -> 10.1007/s12038-020-0025-x
Position fetched: 11547 -> 10.1007/s12039-010-0094-8
Position fetched: 11548 -> 10.1007/s12039-015-0982-z
Position fetched: 11549 -> 10.1007/s12040-020-01475-0
Position fetched: 11550 -> 10.1007/s12041-020-01217-7
Position fetched: 11551 -> 10.1007/s12041-020-01233-7
Position fetched: 11552 -> 10.1007/s12045-020-0982-2
Position fetched: 11553 -> 10.1007/s12045-020-1039-2
Position fetched: 11554 -> 10.1007/s12045-020-1095-7
Position fetched: 11555 -> 10.1007/s12052-007-0002-y
Position fetched: 11556 -> 10.1007/s12052-008-0036-9
Position fetched: 11557 -> 10.1007/s12052-009-0144-1
Position fetched: 11558 -> 10.1007/s12052-010-0290-5
Position fetched: 11559 -> 10.1007/s12

Position fetched: 11694 -> 10.1007/s12109-020-09771-9
Position fetched: 11695 -> 10.1007/s12111-020-09508-0
Position fetched: 11696 -> 10.1007/s12115-012-9584-2
Position fetched: 11697 -> 10.1007/s12115-020-00503-5
Position fetched: 11698 -> 10.1007/s12115-020-00510-6
Position fetched: 11699 -> 10.1007/s12115-020-00521-3
Position fetched: 11700 -> 10.1007/s12115-020-00526-y
Position fetched: 11701 -> 10.1007/s12115-020-00531-1
Position fetched: 11702 -> 10.1007/s12115-020-00537-9
Position fetched: 11703 -> 10.1007/s12115-020-00538-8
Position fetched: 11704 -> 10.1007/s12115-020-00544-w
Position fetched: 11705 -> 10.1007/s12117-020-09398-4
Position fetched: 11706 -> 10.1007/s12117-020-09399-3
Position fetched: 11707 -> 10.1007/s12117-020-09401-y
Position fetched: 11708 -> 10.1007/s12119-020-09786-6
Position fetched: 11709 -> 10.1007/s12119-020-09799-1
Position fetched: 11710 -> 10.1007/s12119-020-09812-7
Position fetched: 11711 -> 10.1007/s12122-020-09312-5
Position fetched: 11712 -> 10

Position fetched: 11847 -> 10.1007/s12250-017-3976-9
Position fetched: 11848 -> 10.1007/s12250-017-4021-8
Position fetched: 11849 -> 10.1007/s12250-018-0003-8
Position fetched: 11850 -> 10.1007/s12250-018-0010-9
Position fetched: 11851 -> 10.1007/s12250-018-0013-6
Position fetched: 11852 -> 10.1007/s12250-018-0017-2
Position fetched: 11853 -> 10.1007/s12250-018-0051-0
Position fetched: 11854 -> 10.1007/s12250-018-0076-4
Position fetched: 11855 -> 10.1007/s12250-019-00145-w
Position fetched: 11856 -> 10.1007/s12250-019-00148-7
Position fetched: 11857 -> 10.1007/s12250-019-00157-6
Position fetched: 11858 -> 10.1007/s12250-019-00169-2
Position fetched: 11859 -> 10.1007/s12250-019-00170-9
Position fetched: 11860 -> 10.1007/s12250-019-00175-4
Position fetched: 11861 -> 10.1007/s12250-019-00182-5
Position fetched: 11862 -> 10.1007/s12250-019-00189-y
Position fetched: 11863 -> 10.1007/s12250-019-00190-5
Position fetched: 11864 -> 10.1007/s12250-020-00213-6
Position fetched: 11865 -> 10.1007/s

Position fetched: 12000 -> 10.1007/s12310-020-09396-5
Position fetched: 12001 -> 10.1007/s12310-020-09401-x
Position fetched: 12002 -> 10.1007/s12311-020-01139-1
Position fetched: 12003 -> 10.1007/s12311-020-01184-w
Position fetched: 12004 -> 10.1007/s12312-020-00834-8
Position fetched: 12005 -> 10.1007/s12312-020-00873-1
Position fetched: 12006 -> 10.1007/s12312-020-00890-0
Position fetched: 12007 -> 10.1007/s12325-020-01230-3
Position fetched: 12008 -> 10.1007/s12325-020-01235-y
Position fetched: 12009 -> 10.1007/s12325-020-01305-1
Position fetched: 12010 -> 10.1007/s12325-020-01339-5
Position fetched: 12011 -> 10.1007/s12325-020-01347-5
Position fetched: 12012 -> 10.1007/s12325-020-01460-5
Position fetched: 12013 -> 10.1007/s12325-020-01495-8
Position fetched: 12014 -> 10.1007/s12325-020-01502-y
Position fetched: 12015 -> 10.1007/s12325-020-01510-y
Position fetched: 12016 -> 10.1007/s12325-020-01519-3
Position fetched: 12017 -> 10.1007/s12325-020-01539-z
Position fetched: 12018 -> 1

The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [ ]:
def save_new_extra_info(len_df_current_extra_info, upto):
    """
    This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
    As a result, this function returns a range of valid entries up to the given parameter. 
    """
    dict_new_extra_info_saver = dict()
    i = len_df_current_extra_info
    while i < upto:
        #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
        dict_new_extra_info_saver[i] = dict_new_extra_info[i]
        i = i + 1 
    return dict_new_extra_info_saver

In [ ]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    #Prior appending, the dictionary is transformed to a DataFrame
    df_newly_fetched = pd.DataFrame(dic)
    
    #For readability, the DataFrame is transposed
    df_newly_fetched_transposed = df_newly_fetched.T
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row.affiliation is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

The existing and newly fetched information are combined into one DataFrame. 

In [ ]:
df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
df_combined_extra_info

In [ ]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [ ]:
def store_df_columns(df):
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    ser_affiliation.to_pickle('extra_info_affiliation_CS.pkl')
    ser_coredata.to_pickle('extra_info_coredata_CS.pkl')
    return ser_affiliation, ser_coredata

In [ ]:
stored_series = store_df_columns(df_combined_extra_info)
stored_series[0]

In [ ]:
stored_series[1]

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [ ]:
# len_data = len(stored_series[0])
# len_data 

In [ ]:
# ser_doi = pd.Series(doi_counted.index[:len_data])
# ser_doi

In [ ]:
# df_current_extra_info_checker = df_combined_extra_info
# df_current_extra_info_checker['doi'] = ser_doi

In [ ]:
# %%time
# len_df_current_extra_info_checker = len(df_current_extra_info_checker)
# dict_new_extra_info_checker = dict()
# i = 0 
# while i < len_df_current_extra_info_checker: ###################################################### 
#     if df_current_extra_info_checker['affiliation'][i] == None:
#         dict_new_extra_info_checker[i] = AffiliationsFromScopusByDOI(client, ser_doi[i])
#         print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
#     i = i + 1    

In [ ]:
# dict_new_extra_info_checker
# -> check if at least one value is not None -> otherwise the process is finished here

In [ ]:
# len(dict_new_extra_info_checker)

In [ ]:
# df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
# df_combined_extra_info_fetched_again

In [ ]:
# store_df_columns(df_combined_extra_info_fetched_again)

In [ ]:
# df_combined_extra_info_fetched_again['check_doi'] = ser_doi
# df_combined_extra_info_fetched_again.head(30)